the program requires python3.5+,autograd '1.4' ,torch '1.12.0',pymanopt '2.0',pandas

In [1]:
from importlib_metadata import version
import autograd.numpy as anp
import torch
import pymanopt
import pymanopt.manifolds
import pymanopt.optimizers
import pandas as pd

In [2]:
version('autograd')

'1.4'

In [3]:
version('torch')

'1.12.0'

In [4]:
version('pymanopt')

'2.0.0'

the purpose of the program is to set the parameters $\Pi\,X,r$ and return a projection matrix $v$  
where $r$ is the factor penality.
where

1. $\Pi$ is a numpy vector
2. $X$ is a numpy matrix
3. $r$ positive number or nulle


In [5]:
SUPPORTED_BACKENDS = ("autograd", "numpy","pytorch")#tensorflow will be next version

In [6]:
#OBJECTIVE FUNCTION

def funobj(K,X,I):

    D = K * I
    invD = anp.linalg.inv(D)
    invD_X = invD @ X

    return anp.trace( invD_X.T @ (K * (I-K)) @ invD_X)

def penobj(K,Pi):

    sub = anp.diag(K) - Pi

    return anp.sum(sub * sub)

In [7]:
#EUCLIDIEAN GRADIANT

def fungrad(K,X,I):
    
    D = K * I
    T0 = anp.linalg.inv(D)
    X_2 = X @ X.T
    T1 = T0 @ X_2 @ T0
    T2 = I - K
    T3 = K * T2
    
    return I * (T1 @ (I - 2*T3@T0)) -2*K*T1

def pengrad(K,Dpi,I):

    return 2 * I * (K - Dpi)

In [8]:
#EUCLIDIEAN HESSIAN

def funhess(K,X,I):
    
    X2 = X @ X.T
    D = K * I
    T0 = anp.linalg.inv(D)
    T1 = T0 @ X2 @ T0
    T2 = T0 @ (K * (I-K))
    T3 = (I * T1) @ T0
    T5 = -T2 @ T3
    derive1 = (T4 + T5) * I + T3 * (I-2*K)
    derive2 = ((I - 2 * T0 @ (I * K)) @ T1) * I

    return -2 * (derive1 + derive2)

def penhess(K,Dpi,I):

    return 2 * I

In [9]:
def checking(Pi,X,r):
    #check $\Pi$ and $X$, $r$ respects property 
    if not(isinstance(Pi,anp.ndarray)):
        raise ValueError(f"Pi is not nympy | type of Pi:'{type(Pi)}'")

    if not(isinstance(X,anp.ndarray)):
        raise ValueError(f"Pi is not nympy | type of X:'{type(X)}'")
        
    if not( isinstance(r,int) or isinstance(r,float) ):
        raise ValueError(f"r is not int or float| type of r:'{type(r)}'")


    if not(len(Pi.shape) == 1 ):
        raise ValueError(f"Pi is not vector numpy")

    if not(len(X.shape) == 2 ):
        raise ValueError(f"X dimension is not matrix numpy ")


    if not(X.shape[0] == Pi.shape[0]):
        raise ValueError(f"X and Pi no match | dimension of X :'{X.shape}' and dimension of Pi :'{Pi.shape}' ")

    n = anp.sum(Pi)

    if not( anp.isclose(n%1,0)):
        raise ValueError(f"sum Pi is very different finterger| sum Pi:'{n}'")

    if not(anp.any((0 < Pi) & (Pi < 1))):
        raise ValueError(f"Pi is not vector of probability| the elemeents:'{Pi[~anp.any((0 < Pi) & (Pi < 1))]}'")
        
    if not(r >= 0):
        raise ValueError(f"r is not positive number| signe of r: - ")


    return Pi.shape[0],int(anp.sum(Pi))

In [10]:
def create_cost_derivate(manifold,Pi,X,r,backend):
    
    euclidean_gradient = euclidean_hessian = None

    Dpi = anp.diag(Pi)
    I = anp.eye(N)
    if backend == "autograd":
        @pymanopt.function.autograd(manifold)
        def cost(v):
            K = v @ v.T

            return funobj(K,X,I)  + r * penobj(K,Pi)

    elif backend == "numpy":
        @pymanopt.function.numpy(manifold)
        def cost(v):
            K = v @ v.T
            return funobj(K,X,I) + r * penobj(K,Pi)

        @pymanopt.function.numpy(manifold)
        def euclidean_gradient(v):
            K = v @ v.T
            return 2 * (fungrad(K,X,I) + r * pengrad(K,Dpi,I)) @ v

        @pymanopt.function.numpy(manifold)
        def euclidean_hessian(v,H):
            K = v @ v.T
            return  2 * ((funhess(K,X,I) + r * penhess(K,Dpi,I)) @ (v @ H.T  + H @ v.T) @ v + (fungrad(K,X,I) +  r * penrad(K,Dpi,I)) @ H)

    elif backend == "pytorch":
        Pi_ = torch.from_numpy(Pi)
        X_  = torch.from_numpy(X)
        I_ = torch.from_numpy(I)
        @pymanopt.function.pytorch(manifold)
        def cost(v):
            K = v @ v.T
            D_ = K * I_
            invD = torch.linalg.inv(D_)
            invD_X_ =  invD @ X_

            return torch.trace( invD_X_.T @ (K * (I_-K)) @   invD_X_) + r * torch.sum((torch.diag(K) - Pi_)**2)
    else:
        raise ValueError(f"Unsupported backend '{backend}'")

    return cost,euclidean_gradient,euclidean_hessian

In [27]:
#the examples are samples of couples (N,n) associated with the kernel
#(N,n) : (20,3) (40,6) (60,9)
#q : 1,2,3
def example(N,n,q):
    data1 = pd.read_csv("../sample/X_"+str(N)+'_'+str(n)+".csv")
    data2 = pd.read_csv("../sample/Ppi_X"+str(q)+'_'+str(N)+'_'+str(n)+".csv")
    data3 = pd.read_csv("../sample/K_"+str(N)+'_'+str(n)+".csv",usecols=['V'+str(i) for i in range(1,N+1)])
    data4 = pd.read_csv("../sample/Kr_X"+str(q)+'_'+str(N)+'_'+str(n)+".csv",usecols=['V'+str(i) for i in range(1,N+1)])
    Pi = data1["pi"].to_numpy()
    X = data1['x'+str(q)].to_numpy().reshape((N,1))
    Ppi = data2.to_numpy()
    K = data3.to_numpy()
    K_r = data4.to_numpy()
    return Pi,X,Ppi,K,K_r.real

$P^\Pi,K,K_r$ are kernel for the couple $(N,n)$ and matrix $X$  
such that $f(P^\Pi,X) < f(K_r,X) < f(K,X)$

In [12]:
N = 20
n = 3
q = 1
r = 10**10

In [13]:
Pi,X,Ppi,K,K_r = example(N,n,q)

In [14]:
funobj(Ppi,X,anp.eye(N)),' < ', funobj(K_r,X,anp.eye(N)), ' < ', funobj(K,X,anp.eye(N))

(17.59154006764077, ' < ', 19.571829130864426, ' < ', 86.53372297065677)

In [15]:
anp.linalg.eig(K)[0]

array([ 9.99999964e-01,  1.00000000e+00,  1.00000000e+00, -1.21807207e-15,
        1.07020525e-15,  7.44541711e-16,  6.92700886e-16,  5.80721955e-16,
        4.39744330e-16, -4.78967670e-16,  3.12515139e-16,  2.28306243e-16,
       -3.97218488e-16, -3.20715935e-16, -2.58666998e-16, -1.74656163e-16,
       -1.44252026e-16,  1.20174426e-16,  2.20374633e-17, -3.17814347e-17])

we take the first 3 columns of the eigenvectors for the projection vectors

In [16]:
VP = lambda K,n : anp.linalg.eig(K)[1][:,:n]

In [17]:
v = VP(K_r,n)

In [18]:
funobj(K_r,X,anp.eye(N)) , funobj(v@v.T,X,anp.eye(N).real)

(19.571829130864426, (13.00729361008596+0j))

Normally $f(K_r,X) \approx f(\mathrm{vp}(K_r) \cdot \mathrm{vp}(K_r)^\top,X)$

# test pymanopt

In [19]:
N,n = checking(Pi,X,r)
manifold = pymanopt.manifolds.grassmann.Grassmann(N, n)
cost,euclidean_gradient,euclidean_hessian = create_manifold_cost_derivate(manifold,Pi,X,r,"autograd")
problem = pymanopt.Problem(manifold, cost, euclidean_gradient=euclidean_gradient)
optimizer = pymanopt.optimizers.SteepestDescent()

In [20]:
result = optimizer.run(problem)
vt = result.point
funobj(vt@vt.T,X,anp.eye(N))

Optimizing...
Iteration    Cost                       Gradient norm     
---------    -----------------------    --------------    
   1         +5.9619492765589724e+09    9.71872646e+09    
   2         +1.8811897352647803e+09    5.94564260e+09    
   3         +1.5234414057333784e+09    5.80518042e+09    
   4         +5.5907644114035213e+08    2.80053806e+09    
   5         +2.2594315856482643e+08    2.45229904e+09    
   6         +1.4306051996962461e+08    2.21856014e+09    
   7         +2.9773888310529709e+07    1.17860034e+09    
   8         +2.1951533757670950e+06    1.71055576e+08    
   9         +8.0982520034918177e+05    1.48965378e+08    
  10         +3.4762644869559840e+05    8.85770918e+07    
  11         +1.6670513513612025e+05    5.15751319e+07    
  12         +9.1113516222066450e+04    3.82923430e+07    
  13         +7.5394666325169397e+04    5.17092287e+07    
  14         +3.0321982081983224e+04    2.35086419e+07    
  15         +2.0326134386194866e+04    2.

93.14052345634052

In [21]:
vt@vt.T

array([[ 1.81816380e-01, -8.66605593e-02, -1.31272213e-02,
        -4.11046194e-02, -1.07260357e-01, -1.02655421e-04,
        -9.29409634e-03,  7.12475670e-02,  7.98899878e-02,
        -1.35329670e-01, -9.36981395e-02, -6.11060054e-02,
        -1.33607073e-01,  7.37575157e-02, -1.05472831e-01,
         7.74198503e-02,  1.61342581e-01, -1.23689017e-01,
         5.52021476e-02,  2.68879425e-02],
       [-8.66605593e-02,  1.33610655e-01,  9.63482677e-02,
         1.21244292e-01,  6.69084753e-02, -4.03638013e-02,
        -1.78185418e-02, -1.33858723e-01,  7.54226152e-02,
         5.95274162e-02,  8.88771303e-02, -4.33701610e-02,
        -4.23826210e-02, -6.64680032e-02,  3.48615822e-02,
        -4.50697957e-02, -4.69891039e-02,  4.05048925e-02,
        -1.24385620e-01, -1.10138490e-01],
       [-1.31272213e-02,  9.63482677e-02,  1.17306610e-01,
         9.87138156e-02,  7.24984593e-03, -6.53842968e-02,
         7.06410350e-02, -1.06475339e-01,  8.71133796e-02,
        -2.38679597e-02,  4.4

In [22]:
optimizer.run(problem,initial_point=vt)
""

Optimizing...
Iteration    Cost                       Gradient norm     
---------    -----------------------    --------------    
   1         +9.3140608123025942e+01    4.64969339e+01    
Terminated - min step_size reached after 1 iterations, 0.01 seconds.



''

In [23]:
vt = optimizer.run(problem,initial_point=VP(K,n)).point
print(funobj(vt@vt.T,X,anp.eye(N)))

Optimizing...
Iteration    Cost                       Gradient norm     
---------    -----------------------    --------------    
   1         +2.4849776261379950e+04    2.09650753e+07    
   2         +2.1026366826680089e+04    2.37484980e+07    
   3         +9.0273940241035543e+03    1.39692306e+07    
   4         +4.3186267502565670e+03    1.25510415e+07    
   5         +4.0870249723686607e+02    3.14414709e+06    
   6         +3.5018208396100380e+02    3.25459989e+06    
   7         +1.7154729055665160e+02    1.72367058e+06    
   8         +1.0385076274986885e+02    6.06474056e+05    
   9         +9.7381747215033741e+01    5.70528572e+05    
  10         +9.2383059384530654e+01    4.29126608e+05    
  11         +8.7992585983633361e+01    1.28307584e+05    
  12         +8.7500340383522584e+01    1.71310948e+05    
  13         +8.6894236506045445e+01    8.86759701e+04    
  14         +8.6725993267420549e+01    6.34552499e+04    
  15         +8.6649611398645135e+01    5.

In [62]:
vt = optimizer.run(problem,initial_point=VP(Ppi,n)).point
print(funobj(vt@vt.T,X,anp.eye(N)))

Optimizing...
Iteration    Cost                       Gradient norm     
---------    -----------------------    --------------    
   1         +1.7595471179847916e+01    4.28498553e+03    
   2         +1.7593056281561680e+01    7.21879295e+03    
   3         +1.7592294724846273e+01    5.46559826e+03    
   4         +1.7591539224624988e+01    4.47471008e+02    
   5         +1.7591536291478437e+01    5.78875906e+02    
   6         +1.7591529587357005e+01    1.23495481e+02    
   7         +1.7591528825391915e+01    1.27107641e+02    
   8         +1.7591528627172995e+01    1.58589081e+02    
   9         +1.7591528114799207e+01    6.17820223e+01    
  10         +1.7591527977157970e+01    1.73560096e+02    
  11         +1.7591527547095072e+01    9.94504901e+01    
  12         +1.7591527467590897e+01    1.16353429e+02    
  13         +1.7591527233693846e+01    5.92221042e+01    
  14         +1.7591527203921654e+01    1.11387815e+02    
  15         +1.7591527097812385e+01    8.

In [24]:
vt = optimizer.run(problem,initial_point=VP(K_r,n)).point
print(funobj(vt@vt.T,X,anp.eye(N)))

Optimizing...
Iteration    Cost                       Gradient norm     
---------    -----------------------    --------------    


TypeError: Grad only applies to real scalar-output functions. Try jacobian, elementwise_grad or holomorphic_grad.

for $\mathrm{vp}(K_r)$ pymanopt break

my objective function on Grassman factory $(N,n)$ in python is not continue beacause normally the program return $v$ such that $f(v \cdot v^\top,X) \approx f(P^\Pi,X)$, I will add that in matlab there is no problem

In [25]:
help(optimizer.run)

Help on method run in module pymanopt.optimizers.steepest_descent:

run(problem, *, initial_point=None, reuse_line_searcher=False) -> pymanopt.optimizers.optimizer.OptimizerResult method of pymanopt.optimizers.steepest_descent.SteepestDescent instance
    Run steepest descent algorithm.
    
    Args:
        problem: Pymanopt problem class instance exposing the cost function
            and the manifold to optimize over.
            The class must either
        initial_point: Initial point on the manifold.
            If no value is provided then a starting point will be randomly
            generated.
        reuse_line_searcher: Whether to reuse the previous line searcher.
            Allows to use information from a previous call to
            :meth:`solve`.
    
    Returns:
        Local minimum of the cost function, or the most recent iterate if
        algorithm terminated before convergence.



In [26]:
help(optimizer)

Help on SteepestDescent in module pymanopt.optimizers.steepest_descent object:

class SteepestDescent(pymanopt.optimizers.optimizer.Optimizer)
 |  SteepestDescent(line_searcher=None, *args, **kwargs)
 |  
 |  Riemannian steepest descent algorithm.
 |  
 |  Perform optimization using gradient descent with line search.
 |  This method first computes the gradient of the objective, and then
 |  optimizes by moving in the direction of steepest descent (which is the
 |  opposite direction to the gradient).
 |  
 |  Args:
 |      line_searcher: The line search method.
 |  
 |  Method resolution order:
 |      SteepestDescent
 |      pymanopt.optimizers.optimizer.Optimizer
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, line_searcher=None, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  run(self, problem, *, initial_point=None, reuse_line_searcher=False) -> pymanopt.optimizers.optimizer.OptimizerResult
 |      Run 

I do not see 'argument' : max_itererations ou iteration